This is in a notebook format, because it seemed more convenient to use Collab Pro instead of a proper VM to train a model on a proper GPU. (It wasn't)

# Finetuning

In [1]:
!pip install -q peft transformers datasets einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [41]:
import os

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    get_linear_schedule_with_warmup
)
from peft import (
    get_peft_model,
    LoraConfig,
)
from tqdm import tqdm
import torch

In [60]:
model_name = "Salesforce/codegen-350M-multi"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side='left'
)
model = AutoModelForCausalLM.from_pretrained(model_name)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [8]:
import json

training_data = json.load(open("training_data.json"))

In [62]:
training_data[0]

{'file_path': 'code_train/0x566164696D_mikroansible_TT.yml',
 'full_code': '---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - connection-check\n    - web-proxy-disable\n    - email-setup    \n    - script-fw-check\n    - services\n    - scritp-route-distance-check\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n',
 'cursor_position': 205,
 'code_before': '---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - connection-check\n    - web-proxy-disable\n    - email-setup    \n    - script-fw-check\n    - services\n    - scritp',
 'code_after': '\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n',
 'expected_code': '-route-distance-check'}

In [63]:
prompt_format = """
# Here's a yaml file to offer a completion for
# Lines after the current one
{text_after}
# Lines before the current one
{text_before}
# Completion:
"""

In [64]:
from typing import List
from transformers import PreTrainedModel

def generate_completion(
        prefixes: List[str],
        model: PreTrainedModel,
        device: str = 'cuda',
        max_generated_tokens: int = 64
    ):
    tokenized = tokenizer(prefixes, return_tensors='pt', padding=True)
    tokenized = {k: v.to(device) for k, v in tokenized.items()}

    output = model.generate(
        **tokenized,
        max_new_tokens=max_generated_tokens,
        eos_token_id=tokenizer.eos_token_id
    )
    tokenized = {k: v.cpu() for k, v in tokenized.items()}

    return tokenizer.batch_decode(output, skip_special_tokens=True)

In [65]:
generate_completion(
    prefixes=[
        prompt_format.format(
            text_after=item["code_after"],
            text_before=item["code_before"]
        )
        for item in training_data[:10]
    ],
    model=model.cuda()
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["\n# Here's a yaml file to offer a completion for\n# Lines after the current one\n\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n\n# Lines before the current one\n---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - connection-check\n    - web-proxy-disable\n    - email-setup    \n    - script-fw-check\n    - services\n    - scritp\n# Completion:\n# - hosts: TT\n#   roles:\n#     - connection-check\n#     - web-proxy-disable\n#     - email-setup\n#     - script-fw-check\n#     - services\n#     - scritp\n# - hosts: TT\n#   ",
 "\n# Here's a yaml file to offer a completion for\n# Lines after the current one\n\n    - services\n    - scritp-route-distance-check\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n\n# Lines before the current one\n---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - con

In [66]:
from torch.utils.data import Dataset, DataLoader

class YamlCompletion_dataset(Dataset):
    def __init__(self, data):
        self.data = [
            {
                "prefix": prompt_format.format(
                    text_after=item["code_after"],
                    text_before=item["code_before"]
                ),
                "completion": item["expected_code"]
            }
            for item in data
        ]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [67]:
dataset = YamlCompletion_dataset(training_data)
dataset[0]

{'prefix': "\n# Here's a yaml file to offer a completion for\n# Lines after the current one\n\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n\n# Lines before the current one\n---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - connection-check\n    - web-proxy-disable\n    - email-setup    \n    - script-fw-check\n    - services\n    - scritp\n# Completion:\n",
 'completion': '-route-distance-check'}

In [68]:
def preprocess_batch(samples_batch, tokenizer, device='cuda'):
    completions = [completion for completion in samples_batch['completion']]
    prefix_and_completions = [
        prefix + completion
        for prefix, completion in zip(samples_batch['prefix'], samples_batch['completion'])
    ]
    tokenized_completions = [
        tokenized_text + [tokenizer.pad_token_id]
        for tokenized_text in tokenizer(completions)['input_ids']
    ]
    tokenized_with_prefixes = [
        tokenized_text + [tokenizer.pad_token_id]
        for tokenized_text in tokenizer(prefix_and_completions)['input_ids']
    ]

    tokenized_completions_padded = [
        [-100]*(len(with_prefix) - len(completion)) + completion
        for with_prefix, completion in zip(
            tokenized_with_prefixes,
            tokenized_completions
        )
    ]

    max_length = max(len(text) for text in tokenized_with_prefixes)

    model_inputs = {
        "input_ids": [],
        "labels": [],
        "attention_mask": []
    }

    for text, label in zip(
        tokenized_with_prefixes,
        tokenized_completions_padded
    ):
        padding_length = max_length - len(text)

        model_inputs['input_ids'].append(
            [tokenizer.pad_token_id]*padding_length + text,
        )
        model_inputs['labels'].append(
            [-100]*padding_length + label,
        )
        model_inputs['attention_mask'].append(
            padding_length * [0] + len(text) * [1]
        )
    model_inputs = {
        k: torch.tensor(v, dtype=int).to(device)
        for k,v in model_inputs.items()
    }
    return model_inputs

In [69]:
peft_config = LoraConfig(
    r=32,
    target_modules='all-linear'
)

peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()


lr = 5e-5
num_epochs = 5
batch_size = 16

trainable params: 10,485,760 || all params: 367,198,208 || trainable%: 2.8556


In [70]:
train_dataloader = DataLoader(dataset, batch_size=batch_size)

In [71]:
from transformers import get_linear_schedule_with_warmup

optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [72]:
from tqdm import tqdm

peft_model = peft_model.cuda()

for epoch in range(num_epochs):
    peft_model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        model_inputs = preprocess_batch(batch, tokenizer=tokenizer)
        outputs = peft_model(**model_inputs)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}:\n{train_ppl=}\n{train_epoch_loss=}")
    model.save_pretrained(f"yaml_complete_epoch_{epoch}")

100%|██████████| 2471/2471 [12:56<00:00,  3.18it/s]


epoch=0:
train_ppl=tensor(12.4103, device='cuda:0')
train_epoch_loss=tensor(2.5185, device='cuda:0')


100%|██████████| 2471/2471 [12:55<00:00,  3.19it/s]


epoch=1:
train_ppl=tensor(8.0226, device='cuda:0')
train_epoch_loss=tensor(2.0823, device='cuda:0')


100%|██████████| 2471/2471 [12:54<00:00,  3.19it/s]


epoch=2:
train_ppl=tensor(6.6509, device='cuda:0')
train_epoch_loss=tensor(1.8947, device='cuda:0')


100%|██████████| 2471/2471 [12:54<00:00,  3.19it/s]


epoch=3:
train_ppl=tensor(5.8408, device='cuda:0')
train_epoch_loss=tensor(1.7649, device='cuda:0')


100%|██████████| 2471/2471 [12:54<00:00,  3.19it/s]


epoch=4:
train_ppl=tensor(5.3871, device='cuda:0')
train_epoch_loss=tensor(1.6840, device='cuda:0')


In [73]:
generate_completion(
    prefixes=[
        prompt_format.format(
            text_after=item["code_after"],
            text_before=item["code_before"]
        )
        for item in training_data[:10]
    ],
    model=peft_model.cuda()
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["\n# Here's a yaml file to offer a completion for\n# Lines after the current one\n\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n\n# Lines before the current one\n---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - connection-check\n    - web-proxy-disable\n    - email-setup    \n    - script-fw-check\n    - services\n    - scritp\n# Completion:\ny-disable",
 "\n# Here's a yaml file to offer a completion for\n# Lines after the current one\n\n    - services\n    - scritp-route-distance-check\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n\n# Lines before the current one\n---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - connection-check\n    - web-proxy-disable\n    - email-setup    \n    - script-fw-ch\n# Completion:\nk",
 "\n# Here's a yaml file to offer a completion for\n# Lines after th

In [77]:
peft_model.push_to_hub("yaml_completion", )

HfHubHTTPError:  (Request ID: Root=1-66d893bf-790055e57b084efe6a033a37;60479972-60b5-44fa-9f72-5e177445eeab)

403 Forbidden: You don't have the rights to create a model under the namespace "alexvumnov".
Cannot access content at: https://huggingface.co/api/repos/create.
If you are trying to create or update content,make sure you have a token with the `write` role.

In [83]:
peft_model.save_pretrained("final_model", save_adapter=True, save_config=True)

In [84]:
merged_model = peft_model.merge_and_unload()
merged_model.save_pretrained("final_merged_model")

# Testing and publishing

In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [24]:
new_model = AutoModelForCausalLM.from_pretrained(
    "final_merged_model",
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained(
    "Salesforce/codegen-350M-multi",
    padding_side='left'
)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

In [25]:
from typing import List
from transformers import PreTrainedModel

def generate_completion(
        prefixes: List[str],
        model: PreTrainedModel,
        device: str = 'cuda',
        max_generated_tokens: int = 64
    ):
    tokenized = tokenizer(prefixes, return_tensors='pt', padding=True)
    tokenized = {k: v.to(device) for k, v in tokenized.items()}

    output = model.generate(
        **tokenized,
        max_new_tokens=max_generated_tokens,
        eos_token_id=tokenizer.eos_token_id
    )
    tokenized = {k: v.cpu() for k, v in tokenized.items()}

    return tokenizer.batch_decode(output, skip_special_tokens=True)

In [26]:
import json

training_data = json.load(open("training_data.json"))

In [27]:
prompt_format = """
# Here's a yaml file to offer a completion for
# Lines after the current one
{text_after}
# Lines before the current one
{text_before}
# Completion:
"""

In [28]:
generate_completion(
    prefixes=[
        prompt_format.format(
            text_after=item["code_after"],
            text_before=item["code_before"]
        )
        for item in training_data[:10]
    ],
    model=new_model.cuda()
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


["\n# Here's a yaml file to offer a completion for\n# Lines after the current one\n\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n\n# Lines before the current one\n---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - connection-check\n    - web-proxy-disable\n    - email-setup    \n    - script-fw-check\n    - services\n    - scritp\n# Completion:\ny-disable",
 "\n# Here's a yaml file to offer a completion for\n# Lines after the current one\n\n    - services\n    - scritp-route-distance-check\n    - socks-disable\n    - script-badblock-check\n    - TT-WiFi-Setup\n    - TT-set-dns-servers\n\n# Lines before the current one\n---\n- hosts: TT\n  user: mikroansible\n  gather_facts: no  \n  connection: local\n  roles:\n    - connection-check\n    - web-proxy-disable\n    - email-setup    \n    - script-fw-ch\n# Completion:\nk",
 "\n# Here's a yaml file to offer a completion for\n# Lines after th

In [29]:
new_model.push_to_hub("yaml_completion")

model.safetensors:   0%|          | 0.00/1.43G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alexvumnov/yaml_completion/commit/05587dc57082262a0ed4f828666dccbcbcf26524', commit_message='Upload CodeGenForCausalLM', commit_description='', oid='05587dc57082262a0ed4f828666dccbcbcf26524', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
tokenizer.push_to_hub("yaml_completion")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alexvumnov/yaml_completion/commit/85524fdc362d2fb9a1d2179c1587b9cb43d0b437', commit_message='Upload tokenizer', commit_description='', oid='85524fdc362d2fb9a1d2179c1587b9cb43d0b437', pr_url=None, pr_revision=None, pr_num=None)

In [35]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained("alexvumnov/yaml_completion")
tokenizer = AutoTokenizer.from_pretrained("alexvumnov/yaml_completion", padding='left')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [34]:
prompt_format = """
# Here's a yaml file to offer a completion for
# Lines after the current one
{text_after}
# Lines before the current one
{text_before}
# Completion:
"""

In [50]:
input_prefix = """
name: my_awesome_env
dependencies:

"""

input_suffix = """"""

In [51]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer, device='cuda')

In [52]:
generator(prompt_format.format(text_after=input_suffix, text_before=input_prefix), max_new_tokens=64)

[{'generated_text': "\n# Here's a yaml file to offer a completion for\n# Lines after the current one\n\n# Lines before the current one\n\nname: my_awesome_env\ndependencies:\n\n\n# Completion:\n- deploy"}]

# Quantization

In [57]:
!pip install -U accelerate bitsandbytes

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [2]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model_8_bit = AutoModelForCausalLM.from_pretrained(
    "alexvumnov/yaml_completion",
    device_map='auto',
    quantization_config=quantization_config
)

In [4]:
model_8_bit.get_memory_footprint()

545652736

In [5]:
model_8_bit.push_to_hub("yaml_completion_8bit")

model.safetensors:   0%|          | 0.00/463M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alexvumnov/yaml_completion_8bit/commit/91f4122dbb258b71aabb3db2d18c5448f20ba89b', commit_message='Upload CodeGenForCausalLM', commit_description='', oid='91f4122dbb258b71aabb3db2d18c5448f20ba89b', pr_url=None, pr_revision=None, pr_num=None)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("alexvumnov/yaml_completion", padding='left')
tokenizer.push_to_hub("yaml_completion_8bit")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/alexvumnov/yaml_completion_8bit/commit/ff6e46e097d8f562fbbcd83ffc81881971f51c9a', commit_message='Upload tokenizer', commit_description='', oid='ff6e46e097d8f562fbbcd83ffc81881971f51c9a', pr_url=None, pr_revision=None, pr_num=None)